 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Define-loss-functions" data-toc-modified-id="Define-loss-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define loss functions</a></span></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Define-models" data-toc-modified-id="Define-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define models</a></span></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Training" data-toc-modified-id="Training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Functions,-generators-and-data" data-toc-modified-id="Functions,-generators-and-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Functions, generators and data</a></span></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Training" data-toc-modified-id="Training-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Training</a></span></li></ul></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Only-the-top-part" data-toc-modified-id="Only-the-top-part-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Only the top part</a></span></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Only-the-bottom-part" data-toc-modified-id="Only-the-bottom-part-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Only the bottom part</a></span><ul class="toc-item"><ul class="toc-item"><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Pseudo-labeling" data-toc-modified-id="Pseudo-labeling-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Pseudo labeling</a></span></li></ul></ul></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Is-the-specialised-model-much-better-than-the-common-model" data-toc-modified-id="Is-the-specialised-model-much-better-than-the-common-model-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Is the specialised model much better than the common model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#top-part" data-toc-modified-id="top-part-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>top part</a></span></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#bottom-part" data-toc-modified-id="bottom-part-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>bottom part</a></span></li></ul></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Test-set-predictions" data-toc-modified-id="Test-set-predictions-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Test set predictions</a></span><ul class="toc-item"><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#top-part" data-toc-modified-id="top-part-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>top part</a></span></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#Bottom-part" data-toc-modified-id="Bottom-part-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Bottom part</a></span></li><li><span><a href="http://localhost:8890/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox.ipynb#combining" data-toc-modified-id="combining-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>combining</a></span></li></ul></li></ul></div>

# Load libraries

In [1]:
import cv2
import numpy as np
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import math
import random
import gzip
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Define loss functions

In [3]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

# Define models

In [4]:
def unet_down_one_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [5]:
def unet_max_pool(inputs):
    x = MaxPooling2D((2, 2), strides=(2, 2))(inputs)
    return x

In [6]:
def unet_up_one_block(up_input, down_input, num_filters):
    x = UpSampling2D((2,2))(up_input)
    x = concatenate([down_input, x], axis=3)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [7]:
def get_unet(input_shape = (256, 256, 3),
             num_classes = 1,
             initial_filters = 32,
             central_filters = 1024):
    
    num_filters = initial_filters
    
    out_list    = [Input(shape=input_shape)]
    down_interim_list = []
    
    while num_filters <= central_filters/2:
        x = unet_down_one_block(out_list[-1], num_filters)
        down_interim_list.append(x)
        num_filters = num_filters * 2
        y = unet_max_pool(x)
        out_list.append(y)
    
    x = unet_down_one_block(out_list[-1], num_filters)
    out_list.append(x)
    num_filters = int(num_filters / 2)
    
    while num_filters >= initial_filters:
        x = unet_up_one_block(out_list[-1], down_interim_list.pop(), num_filters)
        out_list.append(x)
        num_filters = int(num_filters / 2)
    
    classify = Conv2D(num_classes, (1,1), activation = 'sigmoid')(out_list[-1])
    
    model = Model(inputs=out_list[0], outputs=classify)
    
    return model

In [8]:
model = get_unet(input_shape=(768,1920,3), initial_filters=8)

In [9]:
model.load_weights('./weights/best_weights_fullres3.hdf5')

# Training

## Functions, generators and data

In [9]:
df_train = pd.read_csv('data/train_masks.csv')

In [10]:
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

In [11]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [12]:
ids_train_split = list(ids_train_split)
ids_valid_split = list(ids_valid_split)

In [13]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image

In [14]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


In [15]:
def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask


In [16]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [17]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

## Training

In [18]:
train_batch_size = 2
val_batch_size   = 8

In [19]:
model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])

In [21]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.44068337353367232, 0.92385880089930328]

In [20]:
max_epochs = 50

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/horiz_cut.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
7200s - loss: 0.0110 - dice_coeff: 0.9955 - val_loss: 0.0090 - val_dice_coeff: 0.9962
Epoch 2/50
7193s - loss: 0.0075 - dice_coeff: 0.9966 - val_loss: 0.0322 - val_dice_coeff: 0.9899
Epoch 3/50
7194s - loss: 0.0071 - dice_coeff: 0.9967 - val_loss: 0.0106 - val_dice_coeff: 0.9956
Epoch 4/50
7197s - loss: 0.0070 - dice_coeff: 0.9968 - val_loss: 0.0085 - val_dice_coeff: 0.9965
Epoch 5/50
7195s - loss: 0.0067 - dice_coeff: 0.9969 - val_loss: 0.0092 - val_dice_coeff: 0.9960
Epoch 6/50
7190s - loss: 0.0065 - dice_coeff: 0.9970 - val_loss: 0.0077 - val_dice_coeff: 0.9966
Epoch 7/50
7191s - loss: 0.0064 - dice_coeff: 0.9970 - val_loss: 0.0108 - val_dice_coeff: 0.9959
Epoch 8/50
7192s - loss: 0.0063 - dice_coeff: 0.9971 - val_loss: 0.0163 - val_dice_coeff: 0.9941
Epoch 9/50
7194s - loss: 0.0061 - dice_coeff: 0.9971 - val_loss: 0.0073 - val_dice_coeff: 0.9968
Epoch 10/50
7194s - loss: 0.0060 - dice_coeff: 0.9972 - val_loss: 0.0203 - val_dice_coeff: 0.9937
Epoch 11/50
7196s - loss: 0.0

In [24]:
history.history

{'dice_coeff': [0.99547641204088855,
  0.99656272997727269,
  0.99672184245299356,
  0.99678479471136372,
  0.99690351407123723,
  0.99695701736782927,
  0.99703081875526933,
  0.99707062179977834,
  0.99713487077403717,
  0.99717587302885125,
  0.9972247313222955,
  0.99726603491007548,
  0.99730846571395082,
  0.99734954277479093,
  0.99738358953368167,
  0.9974205322289057,
  0.99745514269835822,
  0.99750085735203886,
  0.99753383190098788,
  0.99776134332802136,
  0.99783541459123393],
 'loss': [0.010955939916733661,
  0.0075420257114835031,
  0.0071124950638933324,
  0.0070064343237466837,
  0.0066899393252680579,
  0.0065433648472116388,
  0.006368383872883362,
  0.0062758182126259844,
  0.0061206678139087024,
  0.006028082627226445,
  0.0059079252552991634,
  0.0058195561146551257,
  0.0057122325485787313,
  0.0056209550306649484,
  0.0055586429230447777,
  0.0054548307271623936,
  0.0054122150530100457,
  0.0052764827546867236,
  0.0052027109982926371,
  0.0046979848153577974,

# Only the top part

In [38]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [39]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                
                x_batch.append(img1)
                y_batch.append(mask1)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [40]:
train_batch_size = 6
val_batch_size   = 24

In [41]:
model.load_weights('weights/horiz_cut.hdf5')

In [42]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0059809478709734023, 0.99730878869302142]

In [43]:
model.compile(optimizer=RMSprop(lr=0.00001), loss=bce_dice_loss, metrics=[dice_coeff])

In [45]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/horiz_cut_top.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=int(np.ceil(float(len(ids_train_split)) / float(train_batch_size))),
                    epochs=15,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=int(np.ceil(float(len(ids_valid_split)) / float(val_batch_size))))

Epoch 1/15
1585s - loss: 0.0078 - dice_coeff: 0.9967 - val_loss: 0.0058 - val_dice_coeff: 0.9973
Epoch 2/15
1583s - loss: 0.0061 - dice_coeff: 0.9971 - val_loss: 0.0057 - val_dice_coeff: 0.9973
Epoch 3/15
1582s - loss: 0.0060 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9973
Epoch 4/15
1584s - loss: 0.0059 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 5/15
1583s - loss: 0.0059 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 6/15

Epoch 00005: reducing learning rate to 9.999999747378752e-07.
1583s - loss: 0.0059 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 7/15
1583s - loss: 0.0058 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 8/15
1583s - loss: 0.0058 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 00007: early stopping


# Only the bottom part

In [86]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [87]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                
                                
                x_batch.append(img2)
                y_batch.append(mask2)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [88]:
train_batch_size = 6
val_batch_size   = 24

In [41]:
model.load_weights('weights/horiz_cut.hdf5')

In [22]:
model.compile(optimizer=RMSprop(lr=0.00001), loss=bce_dice_loss, metrics=[dice_coeff])

In [ ]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.008009706292674678, 0.99667233603408156]

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/horiz_cut_bottom.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=int(np.ceil(float(len(ids_train_split)) / float(train_batch_size))),
                    epochs=10,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=int(np.ceil(float(len(ids_valid_split)) / float(val_batch_size))))

Epoch 1/10
1582s - loss: 0.0133 - dice_coeff: 0.9953 - val_loss: 0.0078 - val_dice_coeff: 0.9966
Epoch 2/10
1577s - loss: 0.0073 - dice_coeff: 0.9966 - val_loss: 0.0077 - val_dice_coeff: 0.9966
Epoch 3/10
1577s - loss: 0.0071 - dice_coeff: 0.9967 - val_loss: 0.0076 - val_dice_coeff: 0.9967
Epoch 4/10
1578s - loss: 0.0071 - dice_coeff: 0.9968 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 5/10
1576s - loss: 0.0070 - dice_coeff: 0.9968 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 6/10
1577s - loss: 0.0069 - dice_coeff: 0.9968 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 7/10


In [23]:
model.load_weights('./weights/horiz_cut_bottom.hdf5')

In [24]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0074602240515408086, 0.99670403680539088]

In [25]:
model.compile(optimizer=RMSprop(lr=0.000001), loss=bce_dice_loss, metrics=[dice_coeff])

In [26]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/horiz_cut_bottom2.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=int(np.ceil(float(len(ids_train_split)) / float(train_batch_size))),
                    epochs=5,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=int(np.ceil(float(len(ids_valid_split)) / float(val_batch_size))))

Epoch 1/5
1588s - loss: 0.0071 - dice_coeff: 0.9968 - val_loss: 0.0074 - val_dice_coeff: 0.9967
Epoch 2/5
1579s - loss: 0.0069 - dice_coeff: 0.9968 - val_loss: 0.0074 - val_dice_coeff: 0.9967
Epoch 3/5
1578s - loss: 0.0068 - dice_coeff: 0.9968 - val_loss: 0.0074 - val_dice_coeff: 0.9967
Epoch 4/5
1579s - loss: 0.0068 - dice_coeff: 0.9969 - val_loss: 0.0074 - val_dice_coeff: 0.9967
Epoch 5/5

Epoch 00004: reducing learning rate to 9.999999974752428e-08.
1579s - loss: 0.0069 - dice_coeff: 0.9968 - val_loss: 0.0074 - val_dice_coeff: 0.9967


In [76]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.2, 0.2),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0.05, 0.05))
                img, mask = randomHorizontalFlip(img, mask)
                
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [77]:
model.compile(optimizer=RMSprop(lr=0.000001), loss=bce_dice_loss, metrics=[dice_coeff])

In [78]:
model.load_weights('./weights/horiz_cut_bottom2.hdf5')

In [79]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0074096450477853151, 0.9967301567551663]

In [80]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/horiz_cut_bottom3.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=int(np.ceil(float(len(ids_train_split)) / float(train_batch_size))),
                    epochs=10,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=int(np.ceil(float(len(ids_valid_split)) / float(val_batch_size))))

Epoch 1/10
1580s - loss: 0.0111 - dice_coeff: 0.9955 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 2/10
1576s - loss: 0.0084 - dice_coeff: 0.9962 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 3/10
1575s - loss: 0.0082 - dice_coeff: 0.9963 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 4/10
1577s - loss: 0.0080 - dice_coeff: 0.9963 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 5/10

Epoch 00004: reducing learning rate to 9.999999974752428e-08.
1576s - loss: 0.0085 - dice_coeff: 0.9962 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 6/10
1576s - loss: 0.0080 - dice_coeff: 0.9964 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 7/10
1578s - loss: 0.0079 - dice_coeff: 0.9964 - val_loss: 0.0075 - val_dice_coeff: 0.9967
Epoch 00006: early stopping


### Pseudo labeling

In [89]:
model.load_weights('./weights/horiz_cut_bottom2.hdf5')

In [90]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0074096960079634818, 0.9967301085093635]

In [91]:
model.load_weights('./weights/horiz_cut_bottom3.hdf5')

In [92]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0074598992276343947, 0.99669412908479127]

# Is the specialised model much better than the common model

## top part

In [60]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                
                x_batch.append(img1)
                y_batch.append(mask1)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [61]:
model.load_weights('weights/horiz_cut.hdf5')

In [62]:
val_batch_size   = 24

In [63]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0059811870049692557, 0.9973085491034277]

In [64]:
model.load_weights('weights/horiz_cut_top.hdf5')

In [65]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.005567757181332891, 0.99738260032151682]

## bottom part

In [81]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                                
                x_batch.append(img2)
                y_batch.append(mask2)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [82]:
model.load_weights('weights/horiz_cut.hdf5')

In [83]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0080097747970306575, 0.99667226764682704]

In [84]:
model.load_weights('weights/horiz_cut_bottom3.hdf5')

In [85]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.007459977859437231, 0.99669405062681093]

# Test set predictions

## top part

In [9]:
model.load_weights('weights/horiz_cut_top2.hdf5')

In [3]:
from tqdm import tqdm

In [4]:
df_test = pd.read_csv('data/sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

In [5]:
names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))

In [6]:
import bcolz

In [7]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [ ]:
def save_test_predictions(ids_test, fname):
    val_batch_size = 16
    all_preds = []
    for start in range(0, len(ids_test), val_batch_size):
        x_batch = []
        end = min(start + val_batch_size, len(ids_test))
        ids_test_batch = ids_test[start:end]
        for id in ids_test_batch.values:
            img = cv2.imread('data/test/{}.jpg'.format(id))
            img1  = np.copy(img[0:768, :, :])
            img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
            x_batch.append(img1)
        x_batch = np.array(x_batch, np.float32) / 255
        preds = model.predict_on_batch(x_batch)
        preds = np.squeeze(preds, axis=3)
        all_preds.append(preds)
    all_preds = np.concatenate(all_preds, axis=0)
    all_preds = all_preds.astype(np.float16)
    save_array(fname, all_preds)

In [ ]:
for start in tqdm(range(0, len(ids_test), 250)):
    end = min(start + 250, len(ids_test))
    ids_test_batch = ids_test[start:end]
    save_test_predictions(ids_test_batch, './vert-cut-top-preds/batch-' + str(start))

 25%|██▌       | 102/401 [2:22:34<6:55:24, 83.36s/it]

## Bottom part

In [19]:
model.load_weights('weights/horiz_cut_bottom.hdf5')

In [20]:
from tqdm import tqdm

In [21]:
df_test = pd.read_csv('data/sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

In [22]:
names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))

In [24]:
import bcolz

In [25]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [26]:
def save_test_predictions(ids_test, fname):
    val_batch_size = 16
    all_preds = []
    for start in range(0, len(ids_test), val_batch_size):
        x_batch = []
        end = min(start + val_batch_size, len(ids_test))
        ids_test_batch = ids_test[start:end]
        for id in ids_test_batch.values:
            img = cv2.imread('data/test/{}.jpg'.format(id))
            img2  = np.copy(img[512:1280, :, :])
            img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
            x_batch.append(img2)
        x_batch = np.array(x_batch, np.float32) / 255
        preds = model.predict_on_batch(x_batch)
        preds = np.squeeze(preds, axis=3)
        all_preds.append(preds)
    all_preds = np.concatenate(all_preds, axis=0)
    all_preds = all_preds.astype(np.float16)
    save_array(fname, all_preds)

In [ ]:
for start in tqdm(range(0, len(ids_test), 250)):
    end = min(start + 250, len(ids_test))
    ids_test_batch = ids_test[start:end]
    save_test_predictions(ids_test_batch, './vert-cut-bottom-preds/batch-' + str(start))

100%|██████████| 401/401 [9:39:46<00:00, 67.18s/it]  


## combining

In [8]:
# https://www.kaggle.com/stainsby/fast-tested-rle
def run_length_encode(mask):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    inds = mask.flatten()
    runs = np.where(inds[1:] != inds[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle = ' '.join([str(r) for r in runs])
    return rle


rles = []

In [9]:
for start in tqdm(range(0, len(ids_test), 250)):
    end = min(start + 250, len(ids_test))
    ids_test_batch = ids_test[start:end]
    
    top_part = load_array('./vert-cut-top-preds/batch-'+str(start))
    bottom_part = load_array('./vert-cut-bottom-preds/batch-'+str(start))
    
    top_part = top_part[:,:,2:]
    bottom_part = bottom_part[:,:,0:1918]
    
    part_1 = top_part[:,0:512,:]
    part_2_1 = top_part[:,512:,:]
    part_2_2 = bottom_part[:,0:256,:]
    part_3 = bottom_part[:,256:,:]
    part_2 = np.add(part_2_1, part_2_2)/2
    all_parts = np.concatenate((part_1, part_2, part_3), axis=1)
    all_parts = all_parts > 0.5
    
    for i in range(250):
        mask = all_parts[i,:,:]
        rle = run_length_encode(mask)
        rles.append(rle)

100%|█████████▉| 400/401 [1:59:43<00:18, 18.10s/it]  

IndexError: index 64 is out of bounds for axis 0 with size 64

In [12]:
print("Generating submission file...")
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/submission11.csv.gz', index=False, compression='gzip')

Generating submission file...
